CORRELATION COEFFICIENT FOR YEARS 2004, 2013, 2023 (WITH ALL POSSIBLE COMBINATIONS)

In [2]:
import numpy as np
from scipy.fft import fft, ifft
import netCDF4 as nc
import matplotlib.pyplot as plt
from scipy import stats

def Analyse(f1, f2, Year_label, Latitude_range,arr):
    fill_value = -32767.0
    print(Year_label)
    print(Latitude_range)
    
    nc_file_1 = nc.Dataset(f1, 'r')
    Raw_chlor_1 = np.array(nc_file_1.variables['chlor_a'][:])
    chlor_1 = np.where(Raw_chlor_1 == fill_value, np.nan, Raw_chlor_1)
    latitudes = np.array(nc_file_1.variables['lat'][:])
    longitudes = np.array(nc_file_1.variables['lon'][:])
    nc_file_1.close()

    nc_file_2 = nc.Dataset(f2, 'r')
    Raw_chlor_2 = np.array(nc_file_2.variables['chlor_a'][:])
    chlor_2 = np.where(Raw_chlor_2 == fill_value, np.nan, Raw_chlor_2)
    nc_file_2.close()

    lat_indices = np.where((latitudes >= Latitude_range[0]) & (latitudes <= Latitude_range[1]))[0]
    lon_indices = np.where((longitudes >= 30) & (longitudes <= 120))[0]
    
    chlor_1 = chlor_1[lat_indices, :][:, lon_indices]
    chlor_2 = chlor_2[lat_indices, :][:, lon_indices]
    
    mask = ~np.isnan(chlor_1) & ~np.isnan(chlor_2)
    chlor_1 = chlor_1[mask]
    chlor_2 = chlor_2[mask]
    
    mean_1 = np.mean(chlor_1)
    mean_2 = np.mean(chlor_2)
    chlor_1_Anomaly = chlor_1 - mean_1
    chlor_2_Anomaly = chlor_2 - mean_2

    print(chlor_1.max())    
    correlation_matrix = np.corrcoef(chlor_1_Anomaly, chlor_2_Anomaly)
    correlation_coefficient = correlation_matrix[0, 1]
    print("Correlation Coefficient:", correlation_coefficient)
    
    def cross_correlation_fft(data1, data2):
        n = len(data1) + len(data2) - 1
        f_data1 = fft(data1, n=n)
        f_data2 = fft(data2, n=n)
        cross_corr = ifft(f_data1 * np.conj(f_data2)).real
        return cross_corr
    
    data1 = chlor_1_Anomaly
    data2 = chlor_2_Anomaly
    
    cross_corr = cross_correlation_fft(data1, data2)
    n = len(cross_corr)
    p = cross_corr[(n // 2) + 1:]
    p = np.append(p, cross_corr[:(n // 2) + 1])
    cross_corr = p
    lags = np.arange(-len(data1) + 1, len(data1))
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(data1, data2)
    #print(f"Slope: {slope}")
    #print(f"Intercept: {intercept}")
    
    return correlation_coefficient

corr_arr=np.array([])
Year_label="2004-2013"
cross_arr=Analyse('AQUA_MODIS.20040101_20041231.L3m.YR.CHL.chlor_a.4km.nc','AQUA_MODIS.20130101_20131231.L3m.YR.CHL.chlor_a.4km.nc',"2004-2013",(-30,30) ,corr_arr)
for i in range(-30, 21, 10):
    corr_arr=np.append(corr_arr,Analyse('AQUA_MODIS.20040101_20041231.L3m.YR.CHL.chlor_a.4km.nc', 'AQUA_MODIS.20130101_20131231.L3m.YR.CHL.chlor_a.4km.nc', Year_label, (i, i+10) ,corr_arr))

Year_label="2004-2023"
cross_arr=Analyse('AQUA_MODIS.20040101_20041231.L3m.YR.CHL.chlor_a.4km.nc','AQUA_MODIS.20230101_20231231.L3m.YR.CHL.chlor_a.4km.nc',"2004-2023",(-30,30) ,corr_arr)
for i in range(-30, 21, 10):
    cross_arr=Analyse('AQUA_MODIS.20040101_20041231.L3m.YR.CHL.chlor_a.4km.nc', 'AQUA_MODIS.20230101_20231231.L3m.YR.CHL.chlor_a.4km.nc', Year_label, (i, i+10) ,corr_arr)

Year_label="2013-2023"
cross_arr=Analyse('AQUA_MODIS.20130101_20131231.L3m.YR.CHL.chlor_a.4km.nc','AQUA_MODIS.20230101_20231231.L3m.YR.CHL.chlor_a.4km.nc',"2013-2023",(-30,30) ,corr_arr)
for i in range(-30, 21, 10):
    cross_arr=Analyse('AQUA_MODIS.20130101_20131231.L3m.YR.CHL.chlor_a.4km.nc', 'AQUA_MODIS.20230101_20231231.L3m.YR.CHL.chlor_a.4km.nc', Year_label, (i, i+10) ,corr_arr)
print(corr_arr)

2004-2013
(-30, 30)
84.01758
Correlation Coefficient: 0.9231328707945589
2004-2013
(-30, -20)
63.689487
Correlation Coefficient: 0.9419996323656653
2004-2013
(-20, -10)
53.23507
Correlation Coefficient: 0.9291412355245876
2004-2013
(-10, 0)
79.55438
Correlation Coefficient: 0.9309329509014723
2004-2013
(0, 10)
84.01758
Correlation Coefficient: 0.9444521264398521
2004-2013
(10, 20)
80.25042
Correlation Coefficient: 0.867879956631231
2004-2013
(20, 30)
70.58684
Correlation Coefficient: 0.9045793744694388
2004-2023
(-30, 30)
86.123146
Correlation Coefficient: 0.8966443640945535
2004-2023
(-30, -20)
33.711544
Correlation Coefficient: 0.9030519267991316
2004-2023
(-20, -10)
46.39192
Correlation Coefficient: 0.9009421113149162
2004-2023
(-10, 0)
86.123146
Correlation Coefficient: 0.9151897336751297
2004-2023
(0, 10)
84.01758
Correlation Coefficient: 0.9224388957035858
2004-2023
(10, 20)
78.51122
Correlation Coefficient: 0.8629786197229702
2004-2023
(20, 30)
70.58684
Correlation Coefficient: 

CORRELATION COEFFICIENT FOR THE YEAR 2013-2023

In [8]:
fill_value = -32767.0
print(Year_label)

f1='AQUA_MODIS.20040101_20041231.L3m.YR.CHL.chlor_a.4km.nc'
f2='AQUA_MODIS.20130101_20131231.L3m.YR.CHL.chlor_a.4km.nc'

nc_file_1 = nc.Dataset(f1, 'r')
Raw_chlor_1 = np.array(nc_file_1.variables['chlor_a'][:])
print(Raw_chlor_1.max())
chlor_1 = np.where(Raw_chlor_1 == fill_value, np.nan, Raw_chlor_1)
latitudes = np.array(nc_file_1.variables['lat'][:])
longitudes = np.array(nc_file_1.variables['lon'][:])
nc_file_1.close()

nc_file_2 = nc.Dataset(f2, 'r')
Raw_chlor_2 = np.array(nc_file_2.variables['chlor_a'][:])
chlor_2 = np.where(Raw_chlor_2 == fill_value, np.nan, Raw_chlor_2)
nc_file_2.close()

lat_indices = np.where((latitudes >= -30) & (latitudes <= 30))[0]
lon_indices = np.where((longitudes >= 30) & (longitudes <= 120))[0]
    
chlor_1 = chlor_1[lat_indices, :][:, lon_indices]
chlor_2 = chlor_2[lat_indices, :][:, lon_indices]
    
mask = ~np.isnan(chlor_1) & ~np.isnan(chlor_2)
chlor_1 = chlor_1[mask]
chlor_2 = chlor_2[mask]
    
mean_1 = np.mean(chlor_1)
mean_2 = np.mean(chlor_2)
chlor_1_Anomaly = chlor_1 - mean_1
chlor_2_Anomaly = chlor_2 - mean_2

print(chlor_1.max())    
correlation_matrix = np.corrcoef(chlor_1_Anomaly, chlor_2_Anomaly)
correlation_coefficient = correlation_matrix[0, 1]
print("Correlation Coefficient:", correlation_coefficient)
    
def cross_correlation_fft(data1, data2):
    n = len(data1) + len(data2) - 1
    f_data1 = fft(data1, n=n)
    f_data2 = fft(data2, n=n)
    cross_corr = ifft(f_data1 * np.conj(f_data2)).real
    return cross_corr
    
data1 = chlor_1_Anomaly
data2 = chlor_2_Anomaly
    
cross_corr = cross_correlation_fft(data1, data2)
n = len(cross_corr)
p = cross_corr[(n // 2) + 1:]
p = np.append(p, cross_corr[:(n // 2) + 1])
cross_corr = p
lags = np.arange(-len(data1) + 1, len(data1))
    
slope, intercept, r_value, p_value, std_err = stats.linregress(data1, data2)
    #print(f"Slope: {slope}")
    #print(f"Intercept: {intercept}")
    


2013-2023
86.123146
84.01758
Correlation Coefficient: 0.9231328707945589
